<a href="https://colab.research.google.com/github/vaibhavyesalwad/Sentiment-Analysis/blob/master/BERT_Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing transformers library (for using BERT)**

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 6.9MB/s 
     |████████████████████████████████| 890kB 16.4MB/s 
     |████████████████████████████████| 1.1MB 24.7MB/s 
     |████████████████████████████████| 2.9MB 36.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c8d15cb31269b8bca7c9513cb1394aa3ec3d635ddb424eaceaa192de59c66bfc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


**Importing necessary libraries**

In [2]:
import pandas as pd
import numpy as np
import spacy
import pickle
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification


**Loading dataset from google drive**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/Airline-Sentiment-Analysis

airline_sentiment_analysis.csv	bert_label.pkl	bert_model.h5
bert_inp.pkl			bert_mask.pkl	tb_bert


In [5]:
!cp /mydrive/Airline-Sentiment-Analysis/airline_sentiment_analysis.csv ./

In [6]:
data = pd.read_csv('airline_sentiment_analysis.csv')
data.head()

,Unnamed: 0,airline_sentiment,text
0,1,positive,@VirginAmerica plus you've added commercials t...
1,3,negative,@VirginAmerica it's really aggressive to blast...
2,4,negative,@VirginAmerica and it's a really big bad thing...
3,5,negative,@VirginAmerica seriously would pay $30 a fligh...
4,6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


**Cleaning texts to create corpus**

In [15]:
nlp = spacy.load('en', disable=['parser', 'ner'])
import re

def clean_text(text, use_pronoun_token=False, lower=True):

  if lower:
    tokens = nlp(text.lower())
  else:
    tokens = nlp(text)

  words = []
  for token in tokens:
    lemma = token.lemma_
                
    # in spacy pronouns(you/me/he/she/his/him/they/them...etc) are lemmatised as '-PRON-'
    if lemma=='-PRON-':

      # if we want to use token PRONOUN in place of pronouns 
      if use_pronoun_token:
        words.append("PRONOUN")
      
      # using original pronouns as it is
      else:
        words.append(str(token))         
    
    # ignoring numbers & lemmas having presence of any other than alphanumeric character
    elif not (re.search("[^a-z0-9]", lemma) or lemma.isnumeric()):
      words.append(lemma)

  corpus = " ".join(words)
  return corpus         

In [16]:
data['corpus']= data['text'].apply(clean_text)

**Let's see cleaned text (corpus)**

In [36]:
for i in range(10):
  print(data['text'][i])
  print(data['corpus'][i])
  print()

@VirginAmerica plus you've added commercials to the experience... tacky.
plus you have add commercial to the experience tacky

@VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
it be really aggressive to blast obnoxious entertainment in your guest face amp they have little recourse

@VirginAmerica and it's a really big bad thing about it
and it be a really big bad thing about it

@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.
it's really the only bad thing about flying VA
seriously would pay a flight for seat that do not have this playing it be really the only bad thing about fly va

@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)
yes nearly every time i fly vx this ear worm will not go away

@virginamerica Well, I didn't…but NOW I DO! :-D
well i but now i do

@VirginAmerica it was amazing, and arrived an hour early. You're too good to m

In [18]:
data['label'] = data['airline_sentiment'].apply(lambda x:int(x=='positive'))

In [ ]:
data.head()

,Unnamed: 0,airline_sentiment,text,corpus,label
0,1,positive,@VirginAmerica plus you've added commercials t...,plus have add commercial to the experience tacky,1
1,3,negative,@VirginAmerica it's really aggressive to blast...,be really aggressive to blast obnoxious entert...,0
2,4,negative,@VirginAmerica and it's a really big bad thing...,and be a really big bad thing about,0
3,5,negative,@VirginAmerica seriously would pay $30 a fligh...,seriously would pay a flight for seat that do ...,0
4,6,positive,"@VirginAmerica yes, nearly every time I fly VX...",yes nearly every time i fly vx this ear worm w...,1


In [39]:
sentences=data['corpus']
labels=data['label']
classes = np.unique(data['airline_sentiment'].values)
num_classes = len(classes)
len(sentences),len(labels), num_classes


(11541, 11541, 2)

**Loading pretrained BERT tokenizer for tokenizing texts & BERT sequence classification model for fine tuning**

In [20]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=num_classes)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Observing cleaned text (corpus) lengths and respective counts**

In [21]:
from collections import Counter
Counter(len(sent.split()) for sent in sentences)

Counter({1: 85,
         2: 154,
         3: 156,
         4: 172,
         5: 220,
         6: 229,
         7: 241,
         8: 258,
         9: 287,
         10: 321,
         11: 333,
         12: 366,
         13: 364,
         14: 433,
         15: 491,
         16: 495,
         17: 532,
         18: 592,
         19: 675,
         20: 749,
         21: 825,
         22: 761,
         23: 752,
         24: 681,
         25: 495,
         26: 390,
         27: 265,
         28: 121,
         29: 66,
         30: 19,
         31: 11,
         32: 2})

**Data preprocessing using BERT tokenizer as needed for BERT classification model**

In [89]:
def data_preprocessing(data, max_length=64):
  """Function transforms cleaned text into input-ids and attention-masks"""
  
  sentences = data['corpus']
  input_ids=[]
  attention_masks=[]


  for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_length,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

  input_ids=np.asarray(input_ids)
  attention_masks=np.array(attention_masks)

  return input_ids, attention_masks


# max sequence length choosing 32 as it is good fit for our cleaned texts (corpus)
input_ids, attention_masks = data_preprocessing(data, max_length=32)
labels=np.array(labels)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


**Pickling input-ids and attention-masks so can be used at any run time**

In [23]:
print('Preparing the pickle file.....')

pickle_inp_path='/mydrive/Airline-Sentiment-Analysis/bert_inp.pkl'
pickle_mask_path='/mydrive/Airline-Sentiment-Analysis/bert_mask.pkl'
pickle_label_path='/mydrive/Airline-Sentiment-Analysis/bert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)



Preparing the pickle file.....
Pickle files saved as  /mydrive/Airline-Sentiment-Analysis/bert_inp.pkl /mydrive/Airline-Sentiment-Analysis/bert_mask.pkl /mydrive/Airline-Sentiment-Analysis/bert_label.pkl


In [24]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))


Loading the saved pickle files..
Input shape (11541, 32) Attention mask shape (11541, 32) Input label shape (11541,)


**Splitting features and labels in train and validation split**

In [25]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (9232, 32) Val input shape (2309, 32)
Train label shape (9232,) Val label shape (2309,)
Train attention mask shape (9232, 32) Val attention mask shape (2309, 32)


**Defining hyper-parameters for training of BERT model and saving model weights in google drive for future use**

In [26]:
log_dir='/mydrive/Airline-Sentiment-Analysis/tb_bert'
model_save_path='/mydrive/Airline-Sentiment-Analysis/bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nBert Model',bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])



Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________

Bert Model None


**Training/fine tuning BERT model**

In [27]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=4,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/4
  1/289 [..............................] - ETA: 0s - loss: 0.6423 - accuracy: 0.7188WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
289/289 [==============================] - 3169s 11s/step - loss: 0.2178 - accuracy: 0.9132 - val_loss: 0.1609 - val_accuracy: 0.9350
Epoch 2/4
289/289 [==============================] - 3135s 11s/step - loss: 0.0998 - accuracy: 0.9647 - val_loss: 0.1528 - val_accuracy: 0.9476
Epoch 3/4
289/289 [==============================] - 3149s 11s/step - loss: 0.0458 - accuracy: 0.9862 - val_loss: 0.1976 - val_accuracy: 0.9437
Epoch 4/4
289/289 [==============================] - 3150s 11s/step - loss: 0.0263 - accuracy: 0.9914 - val_loss: 0.2363 - val_accuracy: 0.9459


**Prediction on our custom input texts using fine-tuned model**

In [86]:
def predict_sentiment(texts):
  """Inferencing on any text dataset"""
  data = pd.DataFrame({'text':texts})
  data['corpus'] = data['text'].apply(clean_text)
  
  input_ids, attention_masks = data_preprocessing(data,max_length=32)
  results = bert_model.predict([input_ids, attention_masks])
  results = np.argmax(results[0], axis=1)
  results = [classes[label] for label in results]
  return results
  

In [87]:
sentences = ["shut up", "yeah fucking amazing", "not good", "you fucking good", "I'm gonna sue you"]

In [93]:
predict_sentiment(texts=sentences)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['negative', 'positive', 'negative', 'positive', 'negative']

**Brief evaluation using metrics in classification report** 

In [90]:
from sklearn.metrics import classification_report

In [94]:
results = bert_model.predict([train_inp, train_mask])
train_inferences = np.argmax(results[0], axis=1)
print("Train set")
print(classification_report(train_label, train_inferences))

Train set
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7346
           1       1.00      1.00      1.00      1886

    accuracy                           1.00      9232
   macro avg       1.00      1.00      1.00      9232
weighted avg       1.00      1.00      1.00      9232



In [95]:
results = bert_model.predict([val_inp, val_mask])
val_inferences = np.argmax(results[0], axis=1)
print("Validation set")
print(classification_report(val_label, val_inferences))

Validation set
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1832
           1       0.88      0.85      0.87       477

    accuracy                           0.95      2309
   macro avg       0.92      0.91      0.92      2309
weighted avg       0.95      0.95      0.95      2309

